## Import packages and scripts

In [2]:
# packages
import numpy             as np 
import matplotlib.pyplot as plt
import pandas            as pd
import yaml, os, logging, sys
pd.set_option("display.max_columns", None)

from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# location of the scripts
sys.path.insert(0, '/fefs/aswg/workspace/juan.jimenez/stereo_analysis/scripts')
import auxiliar as aux

# graphic parameters
aux.params()

source_name = 'Crab'

# common data file path
common_data_path = f'/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_data{source_name}.txt'

## Runs selection and configuration
Reading the common runs for MAGIC and LST from a `.txt` file created with other script. The jobs are distributed for each LST-1 run, that will be compared with all MAGIC runs for all the night.

In [3]:
# first of all we can read the common data file
print(f'Opening the file...\n{common_data_path}\n')

# saving the same-night runs comparing LST runs with all MAGIC
jobs_list = np.loadtxt(common_data_path, dtype='str')

print(f'Found {len(jobs_list)} jobs:\n')
for job in jobs_list[:7]:
    print(job)
print('...\n')

# selecting an index for the run you want to analysie
job_index = 1
obs_id_string = jobs_list[job_index]
print(f'Job index selected: {job_index}\nJob chosen:\n{obs_id_string}')

Opening the file...
/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/common_data.txt

Found 27 jobs:

5219-5098298,5098299
5220-5098298,5098299
5229-5098326,5098327
5230-5098326,5098327
5251-5098367,5098368
5252-5098367,5098368
5283-5098498
...

Job index selected: 1
Job chosen:
5220-5098298,5098299


## Finding callibration files of MAGIC and dl1 files form LST
Now we find the files for the selected runs of LST-1 and MAGIC in the specified job
* LST-1 files in `dl1` stage
* MAGIC files in `calib` stage

In [4]:
import find_files

# simply separating the MAGIC and LST runs
obs_id_LST, obs_id_MAGIC = find_files.str_to_run_ids(obs_id_string)
print(f'Selected runs:\nLST run   ---> {obs_id_LST}')
print(f'MAGIC run ---> {obs_id_MAGIC}')

Given only 1 LST run


Selected runs:
LST run   ---> [5220]
MAGIC run ---> [5098298, 5098299]


In [5]:
# finding M1 and M2 callibrated files
files_M1_cal, files_M2_cal = find_files.find_MAGIC_cal(obs_id_MAGIC)

# finding dl1 directories for LST
files_LST_dl1 = find_files.find_LST_dl1(obs_id_LST)

Input MAGIC-I  files path: /fefs/onsite/common/MAGIC/data/M1/event/Calibrated
Input MAGIC-II files path: /fefs/onsite/common/MAGIC/data/M2/event/Calibrated

Finding all runs under all directories...

Filtering the selected obs_ids...
For the MAGIC run 05098298 the number of subruns found are 13
For the MAGIC run 05098299 the number of subruns found are 13

Filtering the selected obs_ids...
For the MAGIC run 05098298 the number of subruns found are 13
For the MAGIC run 05098299 the number of subruns found are 13

Finding all runs for LST...

Selecting the path to the input obs_ids...
For the LST run 05220 the number of subruns found are 131


In [6]:
# opening one of the LST files to see what is inside
print(f'File {files_LST_dl1[0][0]}:')
df = pd.read_hdf(files_LST_dl1[0][0], key='/dl1/event/telescope/parameters/LST_LSTCam')

display(df.head(7))

File /fefs/aswg/data/real/DL1/20210709/v0.9/tailcut84/dl1_LST-1.Run05220.0000.h5:


,obs_id,event_id,intensity,log_intensity,x,y,r,phi,length,length_uncertainty,width,width_uncertainty,psi,skewness,kurtosis,time_gradient,intercept,leakage_intensity_width_1,leakage_intensity_width_2,leakage_pixels_width_1,leakage_pixels_width_2,n_pixels,concentration_cog,concentration_core,concentration_pixel,n_islands,alt_tel,az_tel,calibration_id,dragon_time,ucts_time,tib_time,ucts_jump,mc_type,wl,tel_id,tel_pos_x,tel_pos_y,tel_pos_z,trigger_type,ucts_trigger_type,trigger_time,event_type
0,5220,1,5577.138719,3.746411,-0.014248,-0.010808,0.017883,-2.492635,1.133711,0.007661,1.096413,0.007696,0.298156,0.044128,2.018344,0.287557,14.226729,0.052917,0.127211,0.018868,0.045822,679,0.001343,0.275305,0.003465,41,1.156411,0.867075,0,1.625884e+09,0,0,False,-9999,0.967101,1,50.0,50.0,16.0,1,1,1.625884e+09,32
1,5220,2,88.492595,1.946907,0.278990,-0.382458,0.473402,-0.940567,0.073044,0.004347,0.049255,0.002887,-0.044593,-0.195207,2.253924,-7.303476,7.160019,0.000000,0.000000,0.000000,0.000000,7,0.782795,0.315119,0.315119,1,1.156411,0.867075,0,1.625884e+09,0,0,False,-9999,0.674324,1,50.0,50.0,16.0,1,1,1.625884e+09,32
2,5220,3,50.418310,1.702588,-0.366949,-0.928950,0.998799,-1.946997,0.060763,0.005028,0.042542,0.001771,-1.249941,0.409139,2.380997,-32.144216,8.908218,0.000000,0.000000,0.000000,0.000000,5,0.893095,0.348070,0.348070,1,1.156411,0.867075,0,1.625884e+09,0,0,False,-9999,0.700132,1,50.0,50.0,16.0,1,1,1.625884e+09,32
3,5220,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,-1,1.156411,0.867075,0,1.625884e+09,0,0,False,-9999,NaN,1,50.0,50.0,16.0,1,1,1.625884e+09,32
4,5220,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,-1,1.156411,0.867075,0,1.625884e+09,0,0,False,-9999,NaN,1,50.0,50.0,16.0,1,1,1.625884e+09,32
5,5220,6,204.504253,2.310702,-0.396484,0.207235,0.447377,2.659966,0.097402,0.004296,0.075832,0.002832,-0.124431,0.391302,2.591344,-0.933198,10.784323,0.000000,0.000000,0.000000,0.000000,13,0.495327,0.415934,0.170829,1,1.156412,0.867075,0,1.625884e+09,0,0,False,-9999,0.778547,1,50.0,50.0,16.0,1,1,1.625884e+09,32
6,5220,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-1,NaN,NaN,NaN,-1,1.156412,0.867075,0,1.625884e+09,0,0,False,-9999,NaN,1,50.0,50.0,16.0,32,32,1.625884e+09,2


Some of the MAGIC runs of each night are not observing, so we need to discard those. That the data is not available.

In [7]:
# filtering the empty ones to all data arrays
obs_id_MAGIC = [obsid for obsid, i in zip(obs_id_MAGIC, range(len(files_M1_cal))) if len(files_M1_cal[i]) > 0]        
files_M1_cal = [f     for f,     i in zip(files_M1_cal, range(len(files_M1_cal))) if len(files_M1_cal[i]) > 0] 
files_M2_cal = [f     for f,     i in zip(files_M2_cal, range(len(files_M2_cal))) if len(files_M2_cal[i]) > 0] 

# creating again the obs_id_string
joint_MAGIC_ids =  ','.join(np.array(obs_id_MAGIC).astype(str))
obs_id_string   = f'{obs_id_LST[0]}-{joint_MAGIC_ids}'

print(f'The job with the valid MAGIC runs selected is:\n{obs_id_string}\n')
print(f'Selected runs:\nLST run   ---> {obs_id_LST}')
print(f'MAGIC run ---> {obs_id_MAGIC}')

The job with the valid MAGIC runs selected is:
5220-5098298,5098299

Selected runs:
LST run   ---> [5220]
MAGIC run ---> [5098298, 5098299]


## Configuration
Opening the configuration

In [8]:
# configuration
config_file = f'/fefs/aswg/workspace/juan.jimenez/stereo_analysis/config_files/config{source_name}.yaml'

with open(config_file, 'rb') as f:
    config = yaml.safe_load(f) 

## Convert MAGIC calibrated data to DL1
Creating the `calib` i.e. `.root` files ---> `dl1` as `.h5`

In [9]:
import magicruns_cal_to_dl1

# main output directory
output_dir = '/fefs/aswg/workspace/juan.jimenez/data/dl1/runs_magic_m1_and_m2'
process    = False # false if files aready processed to dl1 set to False

files_M1_dl1 = magicruns_cal_to_dl1.convert(files_M1_cal, obs_id_MAGIC, output_dir, config, 1, process)
files_M2_dl1 = magicruns_cal_to_dl1.convert(files_M2_cal, obs_id_MAGIC, output_dir, config, 2, process)   


calib --> dl1 already done, only extracting filenames (process_data=False)

Converting M1 run 05098298 to DL1...
Converted 13 calibration files to a dl1 file

Converting M1 run 05098299 to DL1...
Converted 13 calibration files to a dl1 file

calib --> dl1 already done, only extracting filenames (process_data=False)

Converting M2 run 05098298 to DL1...
Converted 13 calibration files to a dl1 file

Converting M2 run 05098299 to DL1...
Converted 13 calibration files to a dl1 file


In [10]:
# opening one of the files to see what is inside
print(f'File {files_M1_dl1[0]}:')
df = pd.read_hdf(files_M1_dl1[0], key='events/parameters')

display(df.head(7))

File /fefs/aswg/workspace/juan.jimenez/data/dl1/runs_magic_m1_and_m2/Run05098298/dl1_M1.Run05098298.h5:


,obs_id,event_id,tel_id,pointing_alt,pointing_az,n_pixels,n_islands,time_sec,time_nanosec,time_diff,intensity,skewness,kurtosis,x,y,r,phi,length,length_uncertainty,width,width_uncertainty,psi,intercept,deviation,slope,pixels_width_1,pixels_width_2,intensity_width_1,intensity_width_2
0,5098298,2087,2,1.096836,0.892671,4,1,1625882784,708969600,0.015280,52.812500,-0.001186,2.487316,-0.039883,0.019330,0.044320,154.142479,0.016461,0.001381,0.010206,0.000794,-80.125573,13.767752,0.048600,7.330199,0.000000,0.000000,0.000000,0.000000
1,5098298,2089,2,1.096836,0.892671,142,8,1625882784,716904400,0.003256,917.365885,0.587770,2.302918,-0.031425,-0.067317,0.074291,-115.024357,0.228383,0.004303,0.192234,0.004171,-11.397091,13.924598,1.221540,-1.089540,0.005775,0.011550,0.042598,0.082148
2,5098298,2091,2,1.096837,0.892670,251,1,1625882784,727881200,0.000430,6759.036454,0.385262,2.212963,-0.179596,-0.160950,0.241163,-138.133990,0.172282,0.001154,0.059049,0.000653,1.644552,20.036641,1.200934,12.749365,0.006737,0.012512,0.025161,0.065163
3,5098298,2092,2,1.096837,0.892670,109,5,1625882784,728335800,0.000455,1131.320312,0.493686,2.465290,-0.026978,0.071997,0.076886,110.541140,0.182150,0.003278,0.149181,0.002537,41.303403,15.003413,1.767918,0.313903,0.000000,0.000962,0.000000,0.004047
4,5098298,2093,2,1.096837,0.892670,139,1,1625882784,733570400,0.005235,1707.553386,-0.088703,1.843444,-0.079699,-0.110380,0.136146,-125.830698,0.157610,0.001751,0.060162,0.001216,37.755485,14.446151,0.850373,2.704641,0.000000,0.000000,0.000000,0.000000
5,5098298,2094,2,1.096837,0.892670,7,1,1625882784,740631000,0.006941,72.968750,-0.279411,2.001916,-0.079369,-0.087576,0.118190,-132.185719,0.033537,0.001965,0.010720,0.000686,-35.780434,12.787372,0.613092,9.117930,0.000000,0.000000,0.000000,0.000000
6,5098298,2095,2,1.096837,0.892670,4,1,1625882784,742141400,0.001510,35.578125,0.104165,2.208315,0.148654,0.199447,0.248751,53.301788,0.017367,0.001600,0.010649,0.000901,-21.529553,13.804448,0.163559,5.859585,0.000000,0.000000,0.000000,0.000000


## Merge dl1 files from M1 and M2

In [ ]:
import merge

input_dir  = '/fefs/aswg/workspace/juan.jimenez/data/dl1/runs_magic_m1_and_m2'
output_dir = '/fefs/aswg/workspace/juan.jimenez/data/dl1/nights_magic'
process    = False  # false if files aready merged


files_MAGIC_dl1 = merge.MAGIC_dl1(obs_id_MAGIC, files_M1_dl1, files_M2_dl1, input_dir, output_dir, process)


In [ ]:
# opening one of the files to see what is inside
print(f'File {files_MAGIC_dl1[0]}:')
df = pd.read_hdf(files_MAGIC_dl1[0], key='events/parameters')

display(df.head(7))

## Event coincidence

In [ ]:
config['event_coincidence']

In [ ]:
import event_coincidence_night

output_dir = '/fefs/aswg/workspace/juan.jimenez/data/dl1/joint_runs_magic_lst'
process    = False # if the coincident events already done

files_coinc_dl1 = event_coincidence_night.coincidences(obs_id_LST, files_LST_dl1, files_MAGIC_dl1, output_dir, config, process)


In [ ]:
# opening one of the files to see what is inside
print(f'File {files_coinc_dl1[0][0]}:\n\nCoincidence data:')
df = pd.read_hdf(files_coinc_dl1[0][0], key='coincidence/feature')
display(df)
df = pd.read_hdf(files_coinc_dl1[0][0], key='coincidence/profile')
display(df.head(5))
print('Events data:')
df = pd.read_hdf(files_coinc_dl1[0][0], key='events/parameters')
display(df.head(7))

### Stereo reconstruction

In [ ]:
import reco_stereo

output_dir = '/fefs/aswg/workspace/juan.jimenez/data/dl1/stereo_subruns'
process    = False # you want to generate the stereo files or are already generated

files_stereo_dl1_nomerged = reco_stereo.reconstruct(files_coinc_dl1, obs_id_LST, output_dir, config, process)

In [ ]:
# opening one of the files to see what is inside
print(f'File {files_stereo_dl1_nomerged[0][0]}:')
df = pd.read_hdf(files_stereo_dl1_nomerged[0][0], key='events/parameters')
display(df.head(7))

### Merging to one run file

In [ ]:
import merge

input_dir  = '/fefs/aswg/workspace/juan.jimenez/data/dl1/stereo_subruns'
output_dir = '/fefs/aswg/workspace/juan.jimenez/data/dl1/stereo_runs'
process    = False # you want to merge the files, or if are already merged

files_stereo_dl1 = merge.stereo_dl1(obs_id_LST, files_stereo_dl1_nomerged, input_dir, output_dir, process)

In [ ]:
# opening one of the files to see what is inside
print(f'File {files_stereo_dl1[0]}:')
df = pd.read_hdf(files_stereo_dl1[0], key='events/parameters')

display(df.head(7))

### dl1 to dl2

In [ ]:
import convert_dl1_to_dl2

if source_name == 'Crab':
    RFs_dir = '/fefs/aswg/LST1MAGIC/mc/models/ST0316A/NSB1.5/v01.2/dec_2276' # for CrabNebula
elif source_name == 'BLLac':
    RFs_dir = '/fefs/aswg/LST1MAGIC/mc/models/ST0316A/NSB0.5/v01.2/dec_3476' # for BLLac
elif source_name == 'Mrk421':
    RFs_dir = '/fefs/aswg/LST1MAGIC/mc/models/ST0316A/NSB0.5/v01.2/dec_4822' # for Mrk421
else:
    logger.error(f'The source introduced {source_name} have not specified RFs or is bad written.')
        
output_dir   = '/fefs/aswg/workspace/juan.jimenez/data/dl2/stereo_raw_dl2'
process_data = False

files_stereo_dl2 = convert_dl1_to_dl2.dl1_to_dl2(obs_id_LST, files_stereo_dl1, RFs_dir, output_dir, process_data)

In [ ]:
# opening one of the files to see what is inside
print(f'File {files_stereo_dl2[0]}:')
df = pd.read_hdf(files_stereo_dl2[0], key='events/parameters')

display(df.head(7))